Importing libraries

In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline

Define parameters

In [ ]:
EPISODES = 30000 #For pacman
#EPISODES = 1000 #For CarPole

memory = deque(maxlen=2000)
gamma = 0.99    
epsilon = 1.0 
epsilon_min = 0.01
epsilon_decay = (1-0.01)/30000 #For Pacman
#epsilon_decay = (1-0.01)/1000 #For CartPole
learning_rate = 0.001

Define functions

In [ ]:
def downsample(img):
    return img[::2, ::2]

def resize(img):
    return img[0:170,:]

def preprocess(img):
    return to_grayscale(resize(img))

def change(img):
    img = preprocess(img)
    img = np.reshape(img, (170, 160, 1))
    img = np.expand_dims(img, axis=0)
    return img

def build_model():
        
    # For Cartpole
    #model = Sequential()    
    #model.add(Dense(20, input_dim = state_size , init='uniform', activation='relu'))
    #model.add(Dense(40, activation='relu'))     
    #model.add(Dense(40, activation='relu'))
    #model.add(Dense(20, init='uniform', activation='relu'))
    #model.add(Dense(env.action_space.n, init='uniform', activation='linear'))    
    #model.compile(loss='mean_squared_error',optimizer=Adam(learning_rate))
        
    #return model
       
        
    #For Pacman
    model = Sequential()    
    model.add(Dense(128, input_dim = self.state_size , init='uniform', activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))  
    model.add(Dropout(0.5))    
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, init='uniform', activation='relu'))
    model.add(Dense(env.action_space.n, init='uniform', activation='linear'))    
    model.compile(loss='mean_squared_error',optimizer=Adam(lr=learning_rate))
        
    return model

def get_model_weights():
    # copy weights from model to target_model
    target_model.set_weights(model.get_weights())

def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done)) #Store in memory to do experience replay

def get_action(state):
    if np.random.rand() <= epsilon: #Chance of getting random move
        return random.randrange(action_size)
    actions = model.predict(state)
    return np.argmax(actions[0]) 

def experience_replay(batch_size):
    global epsilon
    minibatch = random.sample(memory, batch_size)
    i = 0
    for state, action, reward, next_state, done in minibatch:
        i = i +1
        target = model.predict(state)
        if done:
            target[0][action] = reward
        else:
            t = target_model.predict(next_state)[0]
            target[0][action] = reward + gamma * np.amax(t)

        model.fit(state, target, epochs=1, verbose = 0)
    if epsilon > epsilon_min:
        epsilon = epsilon - epsilon_decay

def load(name):
    model.load_weights(name)

def save(self, name):
    model.save_weights(name)

Playing

In [ ]:
#env = gym.make('CartPole-v0') # Get the game environment
env = gym.make('MsPacman-ram-v0') # Get the game environment
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

done = False
batch_size = 32
model = build_model()
target_model = build_model()
scores = []

for e in range(EPISODES):
    state = env.reset() #Start a new game
    state = state/255 #If cartPole comment this line
    state = np.reshape(state,[1,state_size])
    tot_reward = 0.0
    done = False
    while not done:
        # env.render()
        action = get_action(state)
        next_state, reward, done, _ = env.step(action)
        if reward == 0:               #
            reward = -0.5             # If Cartpole, comment all these lines
        next_state = next_state/255   #
        
        next_state = np.reshape(next_state,[1,state_size])
        remember(state, action, reward, next_state, done)
        state = next_state
        tot_reward = tot_reward + reward
        if done:
            get_model_weights()
            scores.append(tot_reward)
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, tot_reward, epsilon))
            break
            
    if(len(memory) > batch_size):
        experience_replay(batch_size)
        agent.save("random.h5") #Save the net weights

Saving the graph

pay attention to not overwrite graphs

In [ ]:
plt.figure(figsize=[8,6])
plt.plot(range(len(scores)),scores,linewidth=3.0)
plt.xlabel('Episode ',fontsize=16)
plt.ylabel('Total reward',fontsize=16)
plt.title('Reward per Episode - Pacman',fontsize=16)
plt.savefig('PacmanGraph')

In [ ]:
fram = env.reset()

In [ ]:
plt.imshow(fram)
plt.savefig('Pacmangame')